### Introduction: Comparing Solutions of the 1D Wave Equation with a Smaller PINN Model

---

In this notebook, we will explore the performance of a smaller Physics-Informed Neural Network (PINN) model in solving the 1D wave equation. The focus of this analysis is to understand the capability of the PINN in capturing the dynamics of wave propagation with a reduced model architecture and to compare the predicted results with the analytical solution.

#### Objectives

1. **Loading Pre-Trained Weights**:
   - We will load the pre-trained weights from a smaller PINN model stored as checkpoints on Google Drive. This model has been previously trained on the 1D wave equation dataset, incorporating initial and boundary conditions.

2. **Comparison with Analytical Solution**:
   - The analytical solution of the 1D wave equation provides an exact reference to evaluate the accuracy of the PINN's predictions. We will compare the model's outputs with the analytical solution at various time steps to assess its performance.

3. **Visualization and Analysis**:
   - Visualizations will be used to compare the predicted waveforms and the analytical solution, highlighting any discrepancies or errors. We will also discuss potential reasons for differences and consider the impact of the smaller model's architecture on its predictive accuracy.

#### Key Components

- **Checkpoint Loading**: We will utilize TensorFlow's checkpointing mechanisms to load the model's weights from Google Drive, ensuring that we start from a trained state without re-training.
- **Solution Comparison**: The PINN's predictions will be compared with the analytical solution at multiple time steps, providing insights into the model's strengths and limitations.
- **Model Evaluation**: Metrics such as the L2 norm difference will be used to quantify the deviation of the model's predictions from the analytical solution, offering a clear measure of accuracy.

#### Background

The 1D wave equation, represented by the partial differential equation:

$$
\frac{\partial^2 u}{\partial t^2} = c^2 \frac{\partial^2 u}{\partial x^2}
$$

is a fundamental equation describing wave propagation in a medium. In this analysis, we aim to assess how well a smaller PINN model, with fewer layers and neurons, can approximate the solution to this equation compared to the exact analytical solution.

#### Prerequisites

To follow along, you should be familiar with:
- Basic concepts of neural networks and machine learning.
- Fundamental principles of differential equations, particularly the wave equation.
- Python programming and TensorFlow library usage.

By the end of this notebook, you will have a comprehensive understanding of the performance of smaller PINN models in solving complex PDEs, along with practical insights into model evaluation and comparison with analytical solutions.

In [1]:
# Setup google drive to access the trained weights
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# Weights location
checkpoint_dir = '/content/drive/My Drive/checkpoints_small'

#### Key Considerations
Consistency in Model Architecture: It is crucial to ensure that the architecture of the PINN model used here matches the one employed during training. Any deviations could result in incorrect predictions or failed weight restoration.

#### Reference to Previous Notebook
This notebook builds on the work done in the previous notebook, where the training process and model setup were described in detail. Here, we focus solely on the solution comparison aspect, assuming the model has already been trained and the weights are available.

#### Overview
Problem Parameters: The same physical parameters used during training are defined here, including the length of the string $L$, wave speed $c$, amplitude of the initial displacement $A$, and total time duration $T$.

#### Model Architecture
The neural network architecture is defined to be identical to the one used during training. This includes the number of layers and neurons, activation functions, and output configuration.

#### Analytical Solution
An analytical solution for the 1D wave equation is provided for comparison. This exact solution serves as a benchmark to evaluate the accuracy of the PINN model's predictions.

By maintaining consistency in the model architecture and parameters, and referencing the analytical solution, this notebook provides a focused evaluation of the smaller PINN model's performance in capturing the wave dynamics accurately.

In [3]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

# Define the problem parameters
L = 1.0      # Length of the string
c = 1.0      # Wave speed
A = 0.1      # Amplitude of the initial displacement
T = 1.0      # Total time duration

# Define the neural network architecture (same as the training model)
class PINN(tf.keras.Model):
    def __init__(self):
        super(PINN, self).__init__()
        self.dense1 = tf.keras.layers.Dense(50, activation='tanh')
        self.dense2 = tf.keras.layers.Dense(50, activation='tanh')
        self.dense3 = tf.keras.layers.Dense(50, activation='tanh')
        self.dense4 = tf.keras.layers.Dense(50, activation='tanh')
        self.dense5 = tf.keras.layers.Dense(1, activation=None)

    def call(self, inputs):
        x = self.dense1(inputs)
        x = self.dense2(x)
        x = self.dense3(x)
        x = self.dense4(x)
        return self.dense5(x)

# Create an instance of the PINN model
model = PINN()

# Define the analytical solution for comparison
def analytical_solution(x, t, L, c, A):
    omega = np.pi * c / L
    return A * np.sin(np.pi * x / L) * np.cos(omega * t)

### Restoring the Model Checkpoint

---

To ensure that our solution comparison uses the most up-to-date model, we restore the latest checkpoint saved during training. This step is crucial as it loads the model weights, allowing us to leverage the training progress made previously.

```python
checkpoint = tf.train.Checkpoint(model=model)
manager = tf.train.CheckpointManager(checkpoint, checkpoint_dir, max_to_keep=3)

if manager.latest_checkpoint:
    checkpoint.restore(manager.latest_checkpoint).expect_partial()
    print(f"Model restored from checkpoint: {manager.latest_checkpoint}")
else:
    print("No checkpoint found. Exiting.")
    exit()
```

This code ensures that the model is correctly initialized with the saved weights, ready for the solution comparison against the analytical results. If no checkpoint is found, the script exits to prevent running with uninitialized weights.

In [4]:
# Restore the latest checkpoint
# checkpoint_dir = './checkpoints'
checkpoint = tf.train.Checkpoint(model=model)
manager = tf.train.CheckpointManager(checkpoint, checkpoint_dir, max_to_keep=3)

if manager.latest_checkpoint:
    checkpoint.restore(manager.latest_checkpoint).expect_partial()
    print(f"Model restored from checkpoint: {manager.latest_checkpoint}")
else:
    print("No checkpoint found. Exiting.")
    exit()

Model restored from checkpoint: /content/drive/My Drive/checkpoints_small/ckpt-10
